In [4]:
!pip install transformers torch protobuf tiktoken sentencepiece bitsandbytes==0.43.3 accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 12.4 MB/s eta 0:00:00:00:0100:01


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import math

In [6]:
model_name = "andreidima/Llama-2-7b-Romanian-qlora"

tokenizer = AutoTokenizer.from_pretrained(model_name, device_map='cuda:0')
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='cuda:0')

tokenizer_config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=

In [8]:
def compute_metrics(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    input_ids = inputs["input_ids"]
    
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    
    neg_log_likelihood = outputs.loss
    perplexity = math.exp(neg_log_likelihood.item())
    
    return [perplexity, neg_log_likelihood.item()]

In [10]:
import json
from tqdm import tqdm

In [11]:
regions = [
    'Ardeal',
    'Banat',
    'Bucovina',
    'Canada_EN',
    'Canada_Quebec',
    'Crisana',
    'Dobrogea',
    'Germania',
    'Italia',
    'Maramures',
    'Moldova',
    'Muntenia',
    'Oltenia',
    'Serbia',
    'Spania',
    'Ucraina',
    'UK'
]

In [12]:
import json
from tqdm import tqdm

In [ ]:
results_regions = {}
for region in regions:
    print(region)
    results_regions[region] = []
    results_content = {"perplexity": [], "neg_log_likelihood": []}
    results_titles = {"perplexity": [], "neg_log_likelihood": []}

    with open(f"/kaggle/input/romanianlanguagescrapednews/{region}.json") as f:
        region_json = json.load(f)

    for row in tqdm(region_json):
        row_cnt_result = compute_metrics(model, tokenizer, row['content']  if 'content' in row else row['text'] )
        # print(row_cnt_result)
        results_content['perplexity'].append(row_cnt_result[0])
        results_content['neg_log_likelihood'].append(row_cnt_result[1])
        
        row_title_result = compute_metrics(model, tokenizer, row['title'] )
        results_titles['perplexity'].append(row_title_result[0])
        results_titles['neg_log_likelihood'].append(row_title_result[1])

    perp_content_mean = math.exp(np.array(results_content['neg_log_likelihood']).mean())
    perp_titles_mean = math.exp(np.array(results_titles['neg_log_likelihood']).mean())
    
    results_regions[region].append({'content': results_content, 'titles': results_titles, 'perp_mean_content': perp_content_mean, 'perp_mean_titles': perp_titles_mean})

Ardeal


  2%|▏         | 29/1542 [00:44<39:34,  1.57s/it]

In [ ]:
with open("results_regions.json", f):
    json.dump(results_regions, f, indent=4)